<a href="https://colab.research.google.com/github/akitim/SkillFactory_current/blob/main/Unit_3_ML_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PYTHON-11. Feature Engineering

In [ ]:
import pandas as pd
#import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
import warnings; warnings.filterwarnings('ignore') #(action='once')
import missingno as msno
# from itertools import combinations
# from scipy.stats import ttest_ind
# from scipy.stats import norm
 
%matplotlib inline
 
pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок
pd.options.display.precision = 3
 
 
#import sys
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#sys.path.clear
#sys.path.append('/content/gdrive/MyDrive/SkillFactory')

Mounted at /content/gdrive


#Загрузка

In [30]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
#log = pd.read_csv('/content/gdrive/MyDrive/SkillFactory/log.csv', names=cols)
log = pd.read_csv(log_path, names=cols)
 
#users = pd.read_csv(
#    '/content/gdrive/MyDrive/SkillFactory/vsers.csv',
#    encoding='koi8-r',
#    sep = '\t'
#)
 
users_path = 'https://raw.githubusercontent.com/akitim/datasets/main/users.csv'
users = pd.read_csv(
    users_path,
    encoding='koi8-r',
    sep = '\t'
)
users.columns = ['user_id', 'email', 'geo']
users.sample(1)
#log.sample(1)

,user_id,email,geo
54,User_995,Latinarysa@rambler.ru,Ярославль


Задание 1-5

In [16]:
len(log.loc[log.loc[:, 'time'].isna(), 'time'])

15

In [31]:
#log.dropna(axis=1)
 
#log.dropna(axis=0)
 
#log.loc[users.index.intersection(['time', 'user_id'])].dropna(axis=1)
log.loc[:,'time'].dropna(axis=1)
log.loc[:,'user_id'].dropna(axis=1)

ValueError: ignored